# Генерация датасетов для обучения LLM

Этот notebook показывает как генерировать датасеты для SFT (Supervised Fine-Tuning) обучения.

**Возможности:**
- 58 типов задач (40 математических + 18 физических)
- Новые задачи на reasoning: Судоку, Загадки Эйнштейна, Переправы, Ханойская башня, Кувшины, Игра Ним
- Двуязычность (ru/en)
- 10 уровней сложности
- Форматы: SFT (Alpaca), Chat, JSON, JSONL

In [11]:
# Импорты
from re_rl.dataset_generator import DatasetGenerator
from re_rl.tasks import (
    ArithmeticTask, QuadraticTask, LinearTask,
    KinematicsTask, QuantumTask, NuclearTask,
    ALL_TASK_GENERATORS, ALL_PHYSICS_TASK_GENERATORS
)

# Новые задачи на reasoning
from re_rl.tasks.math.logic import SudokuTask, ZebraPuzzleTask
from re_rl.tasks.math.planning import RiverCrossingTask, TowerOfHanoiTask, WaterJugTask
from re_rl.tasks.math.discrete import NimGameTask

## 1. Быстрый старт: генерация SFT датасета

In [ ]:
generator = DatasetGenerator()

# Генерация SFT датасета
dataset = generator.generate_sft_dataset(
    task_types=["quadratic", "kinematics", "quantum"],
    num_samples=1000,
    language="ru",
    difficulties=[3, 5, 7],
)

print(f"Сгенерировано: {len(dataset)} примеров")
print(f"\nПример:")
print(f"instruction: {dataset[0]['instruction']}")
print(f"input: {dataset[0]['input'][:100]}...")
print(f"output: {dataset[0]['output'][:200]}...")

Сгенерировано: 99 примеров

Пример:
instruction: Решите задачу пошагово, объясняя каждый шаг рассуждения.
input: Фотон с частотой 3.458e+16 Гц падает на металл с работой выхода 2.3326693166027916 эВ. Найдите кинет...
output: Шаг 1: Уравнение Эйнштейна для фотоэффекта: hν = A + Eₖ
E_фотона = hν = 6.626e-34 × 3.458e+16 = 2.291e-17 Дж = 143.02 эВ
Eₖ = hν - A = 143.02 - 2.3326693166027916 = 140.69 эВ

Ответ: 140.6902 эВ...


## 2. Все доступные типы задач

In [13]:
print("МАТЕМАТИЧЕСКИЕ ЗАДАЧИ:")
for i, name in enumerate(sorted(ALL_TASK_GENERATORS.keys()), 1):
    print(f"  {i:2}. {name}")
print(f"\nВсего: {len(ALL_TASK_GENERATORS)}")

print("\nФИЗИЧЕСКИЕ ЗАДАЧИ:")
for i, name in enumerate(ALL_PHYSICS_TASK_GENERATORS.keys(), 1):
    print(f"  {i:2}. {name}")
print(f"\nВсего: {len(ALL_PHYSICS_TASK_GENERATORS)}")

print(f"\n=== ИТОГО: {len(ALL_TASK_GENERATORS) + len(ALL_PHYSICS_TASK_GENERATORS)} типов задач ===")

МАТЕМАТИЧЕСКИЕ ЗАДАЧИ:
   1. analogical
   2. arithmetic
   3. calculus
   4. category_theory
   5. combinatorics
   6. complex_number
   7. contradiction
   8. cubic
   9. differential_equation
  10. exponential
  11. financial_math
  12. futoshiki
  13. geometry
  14. graph
  15. group_theory
  16. inequality
  17. integral
  18. knights_knaves
  19. limits
  20. linear
  21. logarithmic
  22. matrix
  23. number_theory
  24. optimization
  25. quadratic
  26. sequence
  27. series
  28. set_logic
  29. statistics
  30. system_linear
  31. text_stats
  32. trigonometry
  33. urn_probability
  34. vector_3d

Всего: 34

ФИЗИЧЕСКИЕ ЗАДАЧИ:
   1. kinematics
   2. dynamics
   3. energy
   4. momentum
   5. circuits
   6. electrostatics
   7. capacitors
   8. gas_laws
   9. heat_transfer
  10. waves
  11. optics
  12. quantum
  13. nuclear
  14. magnetism
  15. relativity
  16. oscillations
  17. fluids
  18. astrophysics

Всего: 18

=== ИТОГО: 52 типов задач ===


## 3. Примеры задач с разной сложностью

In [14]:
print("=" * 60)
print("ArithmeticTask — арифметические задачи")
print("=" * 60)

for difficulty in [1, 5, 10]:
    task = ArithmeticTask(difficulty=difficulty, language="ru")
    result = task.get_result()
    print(f"\nСложность {difficulty}:")
    print(f"  Задача: {result['problem']}")
    print(f"  Ответ: {result['final_answer']}")

ArithmeticTask — арифметические задачи

Сложность 1:
  Задача: Вычислите: 10 + 6
  Ответ: 16

Сложность 5:
  Задача: Вычислите: 46 - (100 - 88)^2
  Ответ: -98

Сложность 10:
  Задача: Вычислите: 324 - 100% × 800 ÷ 819 × 917 + 5/2 × 5/6 × 171 ÷ ((100% + 768) ÷ 10/9 + 371 + 620) - 543 ÷ 286 + 457 ÷ ((10% - 456) × 328 - 298) ÷ 654
  Ответ: -573.413


In [15]:
print("=" * 60)
print("Квантовая механика — фотоэффект")
print("=" * 60)

task = QuantumTask(task_type="photoelectric", difficulty=5, language="ru", detail_level=5)
task.solve()

print(f"\nЗадача: {task.description}")
print(f"\nШаги решения:")
for i, step in enumerate(task.solution_steps, 1):
    print(f"  {i}. {step}")
print(f"\nОтвет: {task.final_answer}")

Квантовая механика — фотоэффект

Задача: Фотон с частотой 4.562e+15 Гц падает на металл с работой выхода 4.955397640190579 эВ. Найдите кинетическую энергию фотоэлектрона.

Шаги решения:
  1. Шаг 1: Уравнение Эйнштейна для фотоэффекта: hν = A + Eₖ
  2. E_фотона = hν = 6.626e-34 × 4.562e+15 = 3.023e-18 Дж = 18.87 эВ
  3. Eₖ = hν - A = 18.87 - 4.955397640190579 = 13.91 эВ

Ответ: 13.9132 эВ


In [16]:
print("=" * 60)
print("Ядерная физика — энергия связи")
print("=" * 60)

task = NuclearTask(task_type="binding_energy", nucleus="He-4", language="ru", detail_level=10)
task.solve()

print(f"\nЗадача: {task.description}")
print(f"\nШаги решения:")
for i, step in enumerate(task.solution_steps, 1):
    print(f"  {i}. {step}")
print(f"\nОтвет: {task.final_answer}")

Ядерная физика — энергия связи

Задача: Найдите энергию связи ядра He-4 (A = 4, Z = 2).

Шаги решения:
  1. Шаг 1: Eсв = Δm·c² = [Z·m_p + (A-Z)·m_n - M]·931.5 МэВ
  2. m_теор = 2×1.007276 + 2×1.008665 = 4.031882 а.е.м.
  3. Δm = 4.031882 - 4.002603 = 0.029279 а.е.м.
  4. Eсв = 0.029279 × 931.5 = 27.27 МэВ
  5. Eсв/A = 27.27/4 = 6.82 МэВ/нуклон

Ответ: Eсв = 27.27 МэВ (6.82 МэВ/нуклон)


## 4. Новые задачи на Reasoning

Задачи, специально разработанные для обучения LLM рассуждениям и RL.

In [ ]:
print("=" * 60)
print("SudokuTask — Судоку (4x4 и 9x9)")
print("=" * 60)

task = SudokuTask(difficulty=3, language="ru", detail_level=3)
result = task.get_result()

print(f"\nЗадача:\n{result['problem'][:500]}...")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:3], 1):
    print(f"  {i}. {step}")
print(f"\nОтвет:\n{result['final_answer']}")

In [ ]:
print("=" * 60)
print("ZebraPuzzleTask — Загадка Эйнштейна")
print("=" * 60)

task = ZebraPuzzleTask(difficulty=3, language="ru", detail_level=3)
result = task.get_result()

print(f"\nЗадача:\n{result['problem'][:600]}...")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:3], 1):
    print(f"  {i}. {step}")
print(f"\nОтвет: {task.answer}")

In [ ]:
print("=" * 60)
print("RiverCrossingTask — Задача о переправе")
print("=" * 60)

task = RiverCrossingTask(difficulty=1, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:5], 1):
    print(f"  {i}. {step}")
print(f"\nОтвет:\n{result['final_answer']}")

In [ ]:
print("=" * 60)
print("TowerOfHanoiTask — Ханойская башня")
print("=" * 60)

task = TowerOfHanoiTask(difficulty=2, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nКоличество дисков: {task.num_disks}")
print(f"Оптимальное количество ходов: 2^{task.num_disks} - 1 = {2**task.num_disks - 1}")
print(f"\nПервые 7 ходов:")
for i, (disk, from_peg, to_peg) in enumerate(task.moves[:7], 1):
    print(f"  {i}. Диск {disk}: {from_peg} → {to_peg}")

In [ ]:
print("=" * 60)
print("WaterJugTask — Задача о кувшинах")
print("=" * 60)

task = WaterJugTask(difficulty=1, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nОбъёмы кувшинов: {task.capacities}")
print(f"Цель: {task.target} литров")
print(f"Разрешима: {task.is_solvable}")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:5], 1):
    print(f"  {i}. {step}")

In [ ]:
print("=" * 60)
print("NimGameTask — Игра Ним (теория игр)")
print("=" * 60)

task = NimGameTask(difficulty=4, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nКучки: {task.piles}")
print(f"Ним-сумма (XOR): {task.nim_sum}")
if task.winning_move:
    pile_idx, take, new_size = task.winning_move
    print(f"Выигрышный ход: взять {take} из кучки {pile_idx + 1}")
else:
    print("Позиция проигрышная")
print(f"\nОтвет:\n{result['final_answer']}")

## 5. Генерация датасета (пример)

In [17]:
# Генерация 1000 примеров для SFT обучения
generator = DatasetGenerator(output_dir="datasets")

dataset = generator.generate_sft_dataset(
    task_types=None,  # Все типы задач
    num_samples=1000,
    language="ru",
    difficulties=[3, 5, 7, 9],  # Средняя и высокая сложность
)

# Разделение на train/eval
train, eval = generator.split_dataset(dataset, train_ratio=0.9)

print(f"Train: {len(train)} примеров")
print(f"Eval: {len(eval)} примеров")

Train: 853 примеров
Eval: 95 примеров


In [18]:
# Сохранение в JSONL формате
generator.save_jsonl(train, "train.jsonl")
generator.save_jsonl(eval, "eval.jsonl")

print("\nФайлы сохранены! Готово для SFT обучения.")

Сохранено 853 примеров в datasets/train.jsonl
Сохранено 95 примеров в datasets/eval.jsonl

Файлы сохранены! Готово для SFT обучения.


## 6. Chat формат (для Llama/ChatML)

In [19]:
# Генерация в chat формате
chat_data = generator.generate_chat_dataset(
    task_types=["quantum", "kinematics"],
    num_samples=10,
    language="ru"
)

print("Chat формат:")
print(chat_data[0])

Chat формат:
{'messages': [{'role': 'user', 'content': 'Решите задачу пошагово, объясняя каждый шаг рассуждения.\n\nФотон с длиной волны 5.387e-08 м рассеивается на свободном электроне под углом 180°. Найдите изменение длины волны.'}, {'role': 'assistant', 'content': "Шаг 1: Формула Комптона: Δλ = λc(1 - cos θ), где λc = h/(m_e·c)\nλc = h/(m_e·c) = 2.426e-12 м\nΔλ = λc(1 - cos 180°) = 4.853e-12 м\nλ' = λ + Δλ = 5.388e-08 м\n\nОтвет: Δλ = 4.853e-12 м"}], 'metadata': {'task_type': 'quantum', 'difficulty': 4, 'language': 'ru'}}


## 7. Статистика датасета

In [20]:
from collections import Counter

# Распределение по типам задач
task_counts = Counter(item['metadata']['task_type'] for item in dataset)

print("Распределение по типам задач:")
for task_type, count in sorted(task_counts.items(), key=lambda x: -x[1])[:10]:
    print(f"  {task_type}: {count}")

# Распределение по сложности
diff_counts = Counter(item['metadata']['difficulty'] for item in dataset)
print("\nРаспределение по сложности:")
for diff, count in sorted(diff_counts.items()):
    print(f"  Сложность {diff}: {count}")

Распределение по типам задач:
  quadratic: 19
  group_theory: 19
  trigonometry: 19
  logarithmic: 19
  calculus: 19
  arithmetic: 19
  waves: 19
  geometry: 19
  statistics: 19
  knights_knaves: 19

Распределение по сложности:
  Сложность 3: 238
  Сложность 5: 230
  Сложность 7: 231
  Сложность 9: 249


## 8. Генерация полного датасета и сохранение в JSONL

При запуске всего ноутбука генерируется полный датасет со всеми типами задач и сохраняется в JSONL формате.

In [ ]:
import os
import json
from datetime import datetime

# Создаём директорию для датасетов
OUTPUT_DIR = "datasets"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Конфигурация генерации
CONFIG = {
    "num_samples_per_type": 50,  # примеров на тип задачи
    "languages": ["ru"],         # языки
    "difficulties": [3, 5, 7, 9], # уровни сложности
    "train_ratio": 0.9,          # доля train
}

print("=" * 60)
print("ГЕНЕРАЦИЯ ПОЛНОГО ДАТАСЕТА")
print("=" * 60)
print(f"\nКонфигурация:")
print(f"  Примеров на тип задачи: {CONFIG['num_samples_per_type']}")
print(f"  Языки: {CONFIG['languages']}")
print(f"  Уровни сложности: {CONFIG['difficulties']}")
print(f"  Train/Eval split: {CONFIG['train_ratio']*100:.0f}%/{(1-CONFIG['train_ratio'])*100:.0f}%")

In [ ]:
# Генератор датасетов
generator = DatasetGenerator(output_dir=OUTPUT_DIR)

# Все типы задач (включая новые reasoning задачи)
all_task_types = list(ALL_TASK_GENERATORS.keys()) + list(ALL_PHYSICS_TASK_GENERATORS.keys())

print(f"\nВсего типов задач: {len(all_task_types)}")
print(f"Ожидаемое количество примеров: ~{len(all_task_types) * CONFIG['num_samples_per_type']}")

# Новые reasoning задачи (уже включены в ALL_TASK_GENERATORS):
reasoning_tasks = ["sudoku", "zebra_puzzle", "river_crossing", "tower_of_hanoi", "water_jug", "nim_game"]
print(f"Reasoning задачи: {reasoning_tasks}")

In [ ]:
import random

# Генерация датасета
print("\nГенерация датасета...")
print(f"Все типы задач: {len(ALL_TASK_GENERATORS) + len(ALL_PHYSICS_TASK_GENERATORS)}")

full_dataset = []

for lang in CONFIG["languages"]:
    print(f"\n  Язык: {lang}")
    
    # Генерация через DatasetGenerator (ВСЕ задачи, включая новые reasoning)
    all_tasks = list(ALL_TASK_GENERATORS.keys()) + list(ALL_PHYSICS_TASK_GENERATORS.keys())
    
    sft_data = generator.generate_sft_dataset(
        task_types=all_tasks,
        num_samples=len(all_tasks) * CONFIG["num_samples_per_type"],
        language=lang,
        difficulties=CONFIG["difficulties"],
    )
    full_dataset.extend(sft_data)
    print(f"    Сгенерировано: {len(sft_data)} примеров")

print(f"\n  Всего сгенерировано: {len(full_dataset)} примеров")

In [ ]:
# Перемешиваем датасет
random.shuffle(full_dataset)

# Разделяем на train/eval
split_idx = int(len(full_dataset) * CONFIG["train_ratio"])
train_data = full_dataset[:split_idx]
eval_data = full_dataset[split_idx:]

print(f"\nРазделение датасета:")
print(f"  Train: {len(train_data)} примеров")
print(f"  Eval: {len(eval_data)} примеров")

In [ ]:
# Сохранение в JSONL
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

train_file = os.path.join(OUTPUT_DIR, f"reasoning_train_{timestamp}.jsonl")
eval_file = os.path.join(OUTPUT_DIR, f"reasoning_eval_{timestamp}.jsonl")

def save_jsonl(data, filepath):
    """Сохраняет данные в JSONL формате."""
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    return filepath

save_jsonl(train_data, train_file)
save_jsonl(eval_data, eval_file)

print(f"\n" + "=" * 60)
print("ДАТАСЕТ СОХРАНЁН!")
print("=" * 60)
print(f"\nФайлы:")
print(f"  Train: {train_file}")
print(f"  Eval: {eval_file}")
print(f"\nСтатистика:")
print(f"  Всего примеров: {len(full_dataset)}")
print(f"  Train: {len(train_data)}")
print(f"  Eval: {len(eval_data)}")

In [ ]:
# Финальная статистика
from collections import Counter

task_counts = Counter(item['metadata']['task_type'] for item in full_dataset)
diff_counts = Counter(item['metadata']['difficulty'] for item in full_dataset)

print("\nРаспределение по типам задач (топ-15):")
for task_type, count in sorted(task_counts.items(), key=lambda x: -x[1])[:15]:
    print(f"  {task_type}: {count}")

print(f"\n  ... и ещё {len(task_counts) - 15} типов задач")

print("\nРаспределение по сложности:")
for diff, count in sorted(diff_counts.items()):
    print(f"  Сложность {diff}: {count}")

print("\n" + "=" * 60)
print("ГОТОВО! Датасет готов для обучения LLM.")
print("=" * 60)